# The Devil and the Coin Flip Game

>You're playing a game with the devil, with your soul at stake. You're sitting at a circular table, and on the table, there are 4 coins, arranged in a diamond, at the 12, 3, 6, and 9 o'clock positions. We'll number those as positions 0, 1, 2, and 3, respectively. Your goal is to get all 4 coins showing heads. You are blindfolded the entire time, and do not know the initial or subsequent state of the coins.

>Your only way of interacting with the coins is to tell the devil the position number(s) of some coins you want flipped. We call this a "move" on your part. The devil will faithfully perform the requested flips, but will first sneakily rotate the table either 0, 1, 2, or 3 quarter-turns, so that the coins are in different positions. You keep
making moves until 4 heads come up.

> Example: You tell the devil to flip positions 0 and 2 (the 12 o'clock and 6 o'clock positions). The devil could leave the table unrotated (or could rotate it a half-turn), and then flip the two coins that you specified. Or the devil could rotate the table a quarter turn in either direction, and then flip the coins that are now in the 12 o'clock and 6 o'clock locations, which are now the two other coins from the ones you specified.  You don't know how much the table was rotated, so you won't know which coins were flipped (and of course you can't see the state of the table before, during, or after the rotating/flipping action).

> What is a shortest sequence of moves that is guaranteed to win, no matter what the initial state of the coins, and no matter what rotations the devil applies?

# Analysis

The hard part is that we are blindfolded. So we don't know the true state of the coins. We need to represent what we do know: the *set of possible states* of the coins. We call this a *belief state*. At the start of the game, each of the four coins could be either heads or tails, so that's 2<sup>4</sup> = 16 possibilities. However, some of these possibilities are just rotations of other possibilities, and since the devil is free to apply any rotation at any time, it makes more sense to collapse these possibilities together. For example, a set of four possibilities, `{'HHHT', 'HHTH', 'HTHH', 'THHH'}`, all correspond to having three heads on the table, and for purposes of the belief state, I will represent this as a single canonical possibility, `{'HHHT'}`. (I arbitrarily chose the
one that comes first in alphabetical order.)

Once we have the notion of a belief state, we can then update the belief state with the player's move, which is a set of positions to flip, such as `{0, 2}`. The updated belief state consists of every coin sequence in the original belief state, rotated in every possible way, and then with the flips applied. 

Note that the game is described as a turn-taking game, but it is equivalent to a open-loop game where the player specifies a complete sequence of moves all at once, and the devil then follows the instructions. So to solve the game, I need to come up with a sequence of moves that ends up in a belief state consisting of just `{'HHHH'}`. I want it to be a shortest path, so a breadth-first search seems reasonable.


# Implementation Choices

Here are the main concepts, and my implementation choices:

- `Coins`: A *coin sequence* is represented as a `str` of four characters, such as `'HTTT'`. 
- `all_coins`: Every possible coin sequence.
- `Belief`: A *belief state* is represented as a `frozenset` of `Coins` (frozen so that it can be hashed in a `set`).
- `rotations`: The function `rotations(coins)` returns the set of all 4 rotations of coins.
- `initial_belief`: The set of possible (canonical) coin sequences at the start of the game.
- `move`: A *move* is a set of positions to flip, such as `{0, 2}`, which means to flip the 12 o'clock and 6 o'clock positions.
- `update`: The function `update(belief, move)` retuns an updated belief state, representing all the possible coin sequences that could result from a devil rotation followed by the specified flip(s).
- `flip`: The function `flip(coins, move)` flips the specified positions within the coin sequence.

In [1]:
from collections import deque, Counter
from itertools   import chain, product, combinations
import random

Coins = ''.join # Function to make a 4-element Coin Sequence, such as 'HHHT'

all_coins = {Coins(x) for x in product('HT', repeat=4)}

def Belief(coinseq):
    "The set of possible coin sequences (canonicalized)."
    return frozenset(min(rotations(coins)) for coins in coinseq)

def rotations(coins): return {coins[r:] + coins[:r] for r in range(4)}

initial_belief = Belief(all_coins)

def update(belief, move):
    "Update belief: consider all rotations, followed by flips of non-winners."
    return Belief((flip(c, move) if c != 'HHHH' else c)
                  for coins in belief
                  for c in rotations(coins))

def flip(coins, move):
    "Flip the coins in the positions specified by the move."
    coins = list(coins) # Need a mutable sequence
    for i in move:
        coins[i] = ('H' if coins[i] == 'T' else 'T')
    return Coins(coins)

Let's try out some of the functions to see if they look right:

In [2]:
flip('HHHT', {0, 2})

'THTT'

In [3]:
rotations('HHHT')

{'HHHT', 'HHTH', 'HTHH', 'THHH'}

In [4]:
all_coins

{'HHHH',
 'HHHT',
 'HHTH',
 'HHTT',
 'HTHH',
 'HTHT',
 'HTTH',
 'HTTT',
 'THHH',
 'THHT',
 'THTH',
 'THTT',
 'TTHH',
 'TTHT',
 'TTTH',
 'TTTT'}

In [5]:
initial_belief

frozenset({'HHHH', 'HHHT', 'HHTT', 'HTHT', 'HTTT', 'TTTT'})

The above says that there are 16 possible coin sequences,  but only 6 of them are distinct after rotations. We can name the 6: all heads, 3 heads, 2 adjacent heads,  2 opposite heads, 1 head, all tails.

In [6]:
update(initial_belief, {0, 1, 2, 3})

frozenset({'HHHH', 'HHHT', 'HHTT', 'HTHT', 'HTTT'})

That says that if we flip all 4 coins, we eliminate the possibility of 4 tails, but the other 5 coin sequences are still possible.

Everything looks good so far. One more thing: we need to find all subsets of the 4 coin positions:

In [7]:
def powerset(sequence): 
    "All subsets of a sequence."
    # See https://docs.python.org/3.6/library/itertools.html#itertools-recipes
    combos = (combinations(sequence, r) for r in range(len(sequence) + 1))
    return [set(x) for x in chain(*combos)]

powerset(range(4))

[set(),
 {0},
 {1},
 {2},
 {3},
 {0, 1},
 {0, 2},
 {0, 3},
 {1, 2},
 {1, 3},
 {2, 3},
 {0, 1, 2},
 {0, 1, 3},
 {0, 2, 3},
 {1, 2, 3},
 {0, 1, 2, 3}]

# Search for a Solution

The function `search` does a breadth-first search starting
at the initial `belief` state and applying a sequences of moves, trying to
find a path that leads to the goal belief state `{'HHHH'}` (meaning that the only possibility is 4 heads).
As is typical for search algorithms, we build a search tree, keeping a queue of tree `nodes` to consider, where each 
node consists of a path (a sequence of moves) and a resulting belief state. We also keep track, in `explored`, of
the states we have already explored, so that we don't have to revisit them.
 

In [8]:
def search(start=initial_belief):
    "Breadth-first search from starting belief state using moves."
    explored = set()
    q = deque([Node([], start)])
    while q:
        (path, belief) = q.popleft()
        if belief == {'HHHH'}:
            return path
        for move in powerset(range(4)):
            belief2 = update(belief, move)
            if belief2 not in explored:
                explored.add(belief2)
                q.append(Node(path + [move], belief2))
                
def Node(path, belief): return (path, belief)

In [9]:
search()

[{0, 1, 2, 3},
 {0, 2},
 {0, 1, 2, 3},
 {0, 1},
 {0, 1, 2, 3},
 {0, 2},
 {0, 1, 2, 3},
 {0},
 {0, 1, 2, 3},
 {0, 2},
 {0, 1, 2, 3},
 {0, 1},
 {0, 1, 2, 3},
 {0, 2},
 {0, 1, 2, 3}]

That's a 15-move sequence that is guaranteed to lead to a win. Do I believe it?  It does appear to work. A colleague did the puzzle and got the same answer. And here's further validation: The function `random_play` takes a sequence of moves and plays it against a devil that chooses randomly:

In [10]:
def random_play(moves):
    "Play moves against a random devil; return the number of moves until win, or None."
    coins = random.choice(list(all_coins))
    if coins == 'HHHH':
        return 0
    for (i, move) in enumerate(moves, 1):
        coins = random.choice(list(rotations(coins)))
        coins = flip(coins, move)
        if coins == 'HHHH': 
            return i

There are 16 coin sequences so let's call `random_play` 16,000 times, and count the results:

In [11]:
moves = search()

Counter(random_play(moves) for _ in range(16000))

Counter({0: 993,
         1: 994,
         2: 969,
         3: 1004,
         4: 992,
         5: 992,
         6: 957,
         7: 1002,
         8: 1021,
         9: 1013,
         10: 1001,
         11: 1023,
         12: 1007,
         13: 1036,
         14: 1026,
         15: 970})

This says that the player always wins. If the player ever lost, there would be an entry for `None` in the Counter.
The remarkable thing, which I can't explain, is that the number of moves taken to win is so evenly distributed: very nearly 1,000 times for each of the numbers from 0 to 15 moves. Can you explain that?